## Добрый день если вы это читаете значит я не успел сделать третью лабу до того как вы её проверите.

In [1]:
# считывает MovieLens
from os import path
import pandas as pd

data_dir = "./ml-latest-small"

def read_csv(filename: str):
    data = pd.read_csv(path.join(data_dir, filename + ".csv"))
    return data

ratings = read_csv("ratings")
movies = read_csv("movies")
tags = read_csv("tags")

In [3]:

movies.head()

,movieId,title,genres,movie_id
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1
1,2,Jumanji (1995),Adventure|Children|Fantasy,2
2,3,Grumpier Old Men (1995),Comedy|Romance,3
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,4
4,5,Father of the Bride Part II (1995),Comedy,5


In [7]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 4 columns):
movieId     9125 non-null int64
title       9125 non-null object
genres      9125 non-null object
movie_id    9125 non-null int16
dtypes: int16(1), int64(1), object(2)
memory usage: 231.8+ KB


In [6]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296 entries, 0 to 1295
Data columns (total 5 columns):
userId       1296 non-null int64
movieId      1296 non-null int64
tag          1296 non-null object
timestamp    1296 non-null int64
tag_id       1296 non-null int16
dtypes: int16(1), int64(3), object(1)
memory usage: 43.1+ KB


In [56]:
movies["movie_id"] = movies["movieId"].astype("category").cat.codes.copy()
tags["tag_id"] = tags["tag"].astype("category").cat.codes.copy()

In [43]:
mov_tag = pd.merge(movies, tags, how = 'left')

In [44]:
mov_tag.describe()

,movieId,movie_id,userId,timestamp,tag_id
count,9732.000000,9732.000000,1296.000000,1.296000e+03,1296.000000
mean,31671.091759,4582.677250,417.026235,1.324337e+09,336.520833
std,40931.884839,2655.799027,142.183440,1.093886e+08,171.388852
min,1.000000,1.000000,15.000000,1.137217e+09,1.000000
25%,2828.750000,2266.750000,346.000000,1.243455e+09,199.750000
50%,6371.500000,4608.500000,431.000000,1.342849e+09,336.000000
75%,58426.750000,6913.250000,547.000000,1.440380e+09,513.000000
max,164979.000000,9125.000000,663.000000,1.476651e+09,582.000000


In [46]:
mov_tag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9732 entries, 0 to 9731
Data columns (total 8 columns):
movieId      9732 non-null int64
title        9732 non-null object
genres       9732 non-null object
movie_id     9732 non-null int16
userId       1296 non-null float64
tag          1296 non-null object
timestamp    1296 non-null float64
tag_id       1296 non-null float64
dtypes: float64(3), int16(1), int64(1), object(3)
memory usage: 627.3+ KB


In [53]:
mov_tag.dropna(inplace=True)
#tag.fillna("None", inplace=True)


In [8]:
# функция, которая красиво печатает информацию о разреженных матрицах
from scipy.sparse import csr_matrix

def sparse_info(sparse_matrix: csr_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [9]:
from scipy.sparse import csr_matrix
import numpy as np


In [ ]:
#last_movie_id = mov_tag["movie_id"].max()
#last_tag_id = mov_tag["movie_id"].max()

In [ ]:
# переводим матрицу взаимодействий фильмов и тегов к разреженному виду
#from scipy.sparse import csr_matrix
#import numpy as np

#movie_x_tag
#import numpy as np
#movie_x_tag = mov_tag[["movie_id", "tag_id"]].as_matrix()
#import numpy as np

#movie_x_tag
#movie_x_tag


In [57]:
data = []
for i in mov_tag["movie_id"]:
    data.append(1)


In [58]:
len(data)

1296

In [34]:
last_movie_id = len(movies["movie_id"])
last_movie_id 

9125

In [35]:
last_tag_id = len(tags["tag_id"].unique())
last_tag_id

582

In [61]:
from scipy.sparse import coo_matrix
import numpy as np
movie_tag_matrix = coo_matrix((
    data,
    (
        mov_tag["movie_id"],
        mov_tag["tag_id"]
    )
))

sparse_info(movie_tag_matrix)

Размерности матрицы: (9126, 583)
Ненулевых элементов в матрице: 1296
Доля ненулевых элементов: 0.0002435880520060491
Среднее значение ненулевых элементов: 1.0
Максимальное значение ненулевых элементов: 1
Минимальное значение ненулевых элементов: 1


In [62]:
test_indices = np.random.choice(
    ratings.index.values,
    replace=False,
    size=int(len(ratings.index.values) * 0.2)
)
test_data = ratings.iloc[test_indices]
train_data = ratings.drop(test_indices)

In [63]:
train_data.shape

(80004, 4)

In [64]:
import time
from implicit.nearest_neighbours import CosineRecommender

model = CosineRecommender()
print("строим матрицу схожести по косинусной мере")
start = time.time()
model.fit(movie_tag_matrix)
print("построили матрицу схожести по косинусной мере за {} секунд".format(
        time.time() - start))
sparse_info(model.similarity)

строим матрицу схожести по косинусной мере
построили матрицу схожести по косинусной мере за 0.02950739860534668 секунд
Размерности матрицы: (9126, 9126)
Ненулевых элементов в матрице: 8368
Доля ненулевых элементов: 0.00010047563108133921
Среднее значение ненулевых элементов: 0.8210259878428978
Максимальное значение ненулевых элементов: 2.0
Минимальное значение ненулевых элементов: 0.0


In [66]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 8244 to 16341
Data columns (total 4 columns):
userId       20000 non-null int64
movieId      20000 non-null int64
rating       20000 non-null float64
timestamp    20000 non-null int64
dtypes: float64(1), int64(3)
memory usage: 781.2 KB


In [68]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80004 entries, 0 to 100003
Data columns (total 4 columns):
userId       80004 non-null int64
movieId      80004 non-null int64
rating       80004 non-null float64
timestamp    80004 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [ ]:
sparse_info(movie_tag_matrix)

In [ ]:
pre = []
print("получаем рекомендации для всех пользователей")
start = time.time()
movie_matrix = movie_tag_matrix.tocsr()
for mov_id in mov_tag["movie_id"]:
    for movie_id_rec, score in model.recommend(movies, movie_tag_matrix):
         pre.append((mov_id, movie_id_rec, score))
print("получили рекомендации для всех пользователей за {} секнуд".format(
        time.time() - start))

In [ ]:
# разобьём наблюдения на тестовую и обучающую выборки
np.random.seed(0)
test_indices = np.random.choice(
    range(user_item_matrix.nnz),
    replace=False,
    size=int(user_item_matrix.nnz * 0.2)
).tolist()
train_data = user_item_matrix.copy()
train_data.data[test_indices] = 0
train_data.eliminate_zeros()
print("размер обучающей выборки: {}".format(train_data.nnz))
test_data = user_item_matrix.copy()
test_data.data[:] = 0
test_data.data[test_indices] = user_item_matrix.data[test_indices]
test_data.eliminate_zeros()
print("размер тестовой выборки: {}".format(test_data.nnz))

In [ ]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, reciprocal_rank

# обучаем абы какую модель
model = LightFM(loss="bpr")
model.fit(
    train_data,
    num_threads=4
)

train_mrr = reciprocal_rank(model, train_data).mean()
test_mrr = reciprocal_rank(model, test_data).mean()
print('MRR: train %.2f, test %.2f.' % (train_mrr, test_mrr))
for k in [5, 10, 15, 20]:
    train_precision = precision_at_k(model, train_data, k=k).mean()
    test_precision = precision_at_k(model, test_data, k=k).mean()
    print('Precision@%d: train %.2f, test %.2f.' % (k, train_precision, test_precision))

In [ ]:
# выберем информацию по жанрам фильмов
movies_genres = ratings[["movieId", "movie_id"]].drop_duplicates().join(
    movies,
    on="movieId",
    rsuffix="codes",
    lsuffix="movies",
    sort=True
).fillna("None")[["movie_id", "genres"]]
movies_genres["genres_set"] = movies_genres["genres"].apply(lambda x: set(x.split("|")))
movies_genres.head()

In [ ]:
# все возможные жанры
from functools import reduce

reduce(lambda acc, ele: acc.union(ele), movies_genres["genres_set"].tolist(), set())

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# приравняем None и (no genres listed)
movies_genres.loc[movies_genres["genres"] == "(no genres listed)", "genres"] = "None"
# уберём все спецсимволы, кроме |
movies_genres["genres"] = movies_genres["genres"].apply(
    lambda x: x.replace("-", "")
)

movies_features = CountVectorizer().fit_transform(movies_genres["genres"])
movies_features

In [ ]:
print(movies_genres["genres"][:1])
print(movies_features[0].todense())

In [ ]:
# добавляем к movie_id ещё и информацию о жанрах
from scipy.sparse import hstack, identity

features = hstack([
    identity(movies_genres.shape[0]),
    movies_features
])

In [ ]:
features

In [ ]:
# обучаем модель с жанрами
hybrid = LightFM(loss="bpr")
hybrid.fit(
    train_data,
    num_threads=4,
    item_features=features
)

train_mrr = auc_score(hybrid, train_data, item_features=features).mean()
test_mrr = auc_score(hybrid, test_data, item_features=features).mean()
print('MRR: train %.2f, test %.2f.' % (train_mrr, test_mrr))
#for k in [5, 10, 15, 20]:
    #train_precision = precision_at_k(hybrid, train_data, item_features=features, k=k).mean()
    #test_precision = precision_at_k(hybrid, test_data, item_features=features, k=k).mean()
#print('Precision@%d: train %.2f, test %.2f.' % (k, train_precision, test_precision))